In [698]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import math

In [280]:
lines = []

for year in [2018, 2019, 2020, 2021, 2022]:
    resp = requests.get(
        'https://results.ittf.link/index.php',
        params=dict(
            option='com_fabrik',
            view='list',
            listid=1,
            Itemid=111,
            resetfilters=0,
            clearordering=0,
            clearfilters=0,
            limitstart1=0,
            limit1=500,
            fab_tournaments___code=year
        )
    )
    soup = BeautifulSoup(resp.text)
    rows = soup.find('table', {'id': 'list_1_com_fabrik_1'}).find_all("tr", {"class": "fabrik_row"})
    for row in rows:
        lines.append(list(map(lambda x: x.text.strip(), row.find_all('td'))))

In [282]:
df = pd.DataFrame(lines, columns=['tournament_id', 'year', 'tournament_name', 'tournament_type', 'tournament_kind', 'organizer', 'matches', 'from', 'to', 'foo'])
df.drop(columns=['foo'], inplace=True)
df

,tournament_id,year,tournament_name,tournament_type,tournament_kind,organizer,matches,from,to
0,2813,2018,"2018 - World Tour, Hungarian Open, Budapest (HUN)",World Tour / Pro Tour,--,HUN,563,2018-01-18,2018-01-21
1,2882,2018,"2018 - ITTF Europe Top 16 Cup, Montreux (SUI)",Continental,--,SUI,32,2018-02-03,2018-02-04
2,2835,2018,"2018 - Oman Junior and Cadet Open, Muscat (OMA)",World Junior Circuit,Premium,OMA,574,2018-02-07,2018-02-11
3,2865,2018,"2018 - YOG, Road to BA Europe, Hodonin (CZE)",Olympic Games,--,CZE,113,2018-02-12,2018-02-13
4,2840,2018,2018 - ITTF Junior Circuit Premium Czech Junio...,World Junior Circuit,Premium,CZE,1534,2018-02-14,2018-02-18
...,...,...,...,...,...,...,...,...,...
372,5370,2022,"2022 - WTT Youth Contender, Sulaymaniyah (IRQ)",WTT Youth Contender Series,WTT Youth Contender,IRQ,403,2022-03-14,2022-03-20
373,5397,2022,"2022 - WTT Contender, Doha (QAT)",WTT Contender Series,WTT Contender,QAT,289,2022-03-18,2022-03-24
374,5380,2022,"2022 - WTT Youth Contender, Tunis (TUN)",WTT Youth Contender Series,WTT Youth Contender,TUN,362,2022-03-21,2022-03-27
375,5400,2022,"2022 - WTT Star Contenders, Doha (QAT)",WTT Contender Series,WTT Star Contender,QAT,267,2022-03-25,2022-03-31


In [283]:
df.to_csv('data/tournaments_with_junior_2018.tsv', index=False, sep='\t')

In [344]:
adult = df[
    ~(
        df.tournament_type.str.contains('Junior')
        | df.tournament_name.str.contains('YOG')
        | df.tournament_name.str.contains('Junior')
        | df.tournament_name.str.contains('Cadet')
        | df.tournament_name.str.contains('Youth')
        | df.tournament_name.str.contains('U21')
        | df.tournament_name.str.contains('U18')
        | df.tournament_name.str.contains('Senior')
    )
].copy()
adult.to_csv('data/tournaments_2018.tsv', index=False, sep='\t')
# adult[adult.matches != ''].matches.astype(int).sort_values()
# adult.loc[351]

In [665]:
games = []
for row in adult.itertuples():
    resp = requests.get(
        'https://results.ittf.link/index.php',
        params=dict(
            option='com_fabrik',
            view='list',
            listid=47,
            Itemid=158,
            resetfilters=1,
            limit47=2000,
            fab_matches___tournament_id=row.tournament_id
        )
    )

    soup = BeautifulSoup(resp.text)
    rows = soup.find('table', {'id': 'list_47_com_fabrik_47'}).find_all("tr", {"class": "fabrik_row"})
    for row in rows:
        *cells, link = row.find_all('td')
        lis = list(map(lambda x: x.text.strip(), cells))
        lis.append(row.tournament_id)
        lis.append(link.find('a')['data-rowid'])
        games.append(lis)


In [288]:
# resp = requests.get(
#     'https://results.ittf.link/index.php',
#     params=dict(
#         option='com_fabrik',
#         view='list',
#         listid=47,
#         Itemid=158,
#         resetfilters=1,
#         limit47=2000,
#         fab_matches___tournament_id=2876
#     )
# )


In [668]:
gamef = pd.DataFrame(games, columns=['tournament_name', 'player_a', 'player_b', 'player_x', 'player_y', 'event_type', 'round', 'res_a', 'res_x', 'winner', 'winner_dbl', 'view', 'foo', 'match_id'], dtype=str)
gamef.drop(columns=['view', 'foo'], inplace=True)
gamef

,tournament_name,player_a,player_b,player_x,player_y,event_type,round,res_a,res_x,winner,winner_dbl,match_id
0,"2018 - World Tour, Hungarian Open, Budapest (HUN)",APOLONIA Tiago (POR) (100621),,BADOWSKI Marek (POL) (120154),,Men's Singles,2,4,1,APOLONIA Tiago,,329372
1,"2018 - World Tour, Hungarian Open, Budapest (HUN)",NUYTINCK Cedric (BEL) (106814),,NAGY Krisztian (HUN) (106542),,Men's Singles,2,4,2,NUYTINCK Cedric,,329373
2,"2018 - World Tour, Hungarian Open, Budapest (HUN)",NUYTINCK Cedric (BEL) (106814),,YEFIMOV Viktor (UKR) (110367),,Men's Singles,3,1,4,YEFIMOV Viktor,,329374
3,"2018 - World Tour, Hungarian Open, Budapest (HUN)",YEFIMOV Viktor (UKR) (110367),,NAGY Krisztian (HUN) (106542),,Men's Singles,1,4,1,YEFIMOV Viktor,,329375
4,"2018 - World Tour, Hungarian Open, Budapest (HUN)",FRANZISKA Patrick (GER) (102832),,CHEN Alexander (AUT) (113914),,Men's Singles,2,4,1,FRANZISKA Patrick,,329376
...,...,...,...,...,...,...,...,...,...,...,...,...
42630,"2022 - WTT Star Contenders, Doha (QAT)",KARLSSON Kristian (SWE) (104379),,GACINA Andrej (CRO) (102891),,Men's Singles,4,0,4,GACINA Andrej,,1018142
42631,"2022 - WTT Star Contenders, Doha (QAT)",ARUNA Quadri (NGR) (112092),,LIM Jonghoon (KOR) (117357),,Men's Singles,4,2,4,LIM Jonghoon,,1018143
42632,"2022 - WTT Star Contenders, Doha (QAT)",LIN Yun-Ju (TPE) (121582),CHENG I-Ching (TPE) (110797),LEBESSON Emmanuel (FRA) (105136),YUAN Jia Nan (FRA) (137894),Mixed Doubles,2,0,3,LEBESSON Emmanuel,YUAN Jia Nan,1018144
42633,"2022 - WTT Star Contenders, Doha (QAT)",HAN Ying (GER) (118762),,KIHARA Miyuu (JPN) (131036),,Women's Singles,2,3,4,KIHARA Miyuu,,1018145


In [669]:
agamef = gamef[gamef.event_type.isin([
    "Men's Singles",
    "Women's Singles",
    "Men's Teams",
    "Women's Teams",
    "Men's Doubles",
    "Women's Doubles",
    "Mixed Doubles",
])]
agamef

,tournament_name,player_a,player_b,player_x,player_y,event_type,round,res_a,res_x,winner,winner_dbl,match_id
0,"2018 - World Tour, Hungarian Open, Budapest (HUN)",APOLONIA Tiago (POR) (100621),,BADOWSKI Marek (POL) (120154),,Men's Singles,2,4,1,APOLONIA Tiago,,329372
1,"2018 - World Tour, Hungarian Open, Budapest (HUN)",NUYTINCK Cedric (BEL) (106814),,NAGY Krisztian (HUN) (106542),,Men's Singles,2,4,2,NUYTINCK Cedric,,329373
2,"2018 - World Tour, Hungarian Open, Budapest (HUN)",NUYTINCK Cedric (BEL) (106814),,YEFIMOV Viktor (UKR) (110367),,Men's Singles,3,1,4,YEFIMOV Viktor,,329374
3,"2018 - World Tour, Hungarian Open, Budapest (HUN)",YEFIMOV Viktor (UKR) (110367),,NAGY Krisztian (HUN) (106542),,Men's Singles,1,4,1,YEFIMOV Viktor,,329375
4,"2018 - World Tour, Hungarian Open, Budapest (HUN)",FRANZISKA Patrick (GER) (102832),,CHEN Alexander (AUT) (113914),,Men's Singles,2,4,1,FRANZISKA Patrick,,329376
...,...,...,...,...,...,...,...,...,...,...,...,...
42630,"2022 - WTT Star Contenders, Doha (QAT)",KARLSSON Kristian (SWE) (104379),,GACINA Andrej (CRO) (102891),,Men's Singles,4,0,4,GACINA Andrej,,1018142
42631,"2022 - WTT Star Contenders, Doha (QAT)",ARUNA Quadri (NGR) (112092),,LIM Jonghoon (KOR) (117357),,Men's Singles,4,2,4,LIM Jonghoon,,1018143
42632,"2022 - WTT Star Contenders, Doha (QAT)",LIN Yun-Ju (TPE) (121582),CHENG I-Ching (TPE) (110797),LEBESSON Emmanuel (FRA) (105136),YUAN Jia Nan (FRA) (137894),Mixed Doubles,2,0,3,LEBESSON Emmanuel,YUAN Jia Nan,1018144
42633,"2022 - WTT Star Contenders, Doha (QAT)",HAN Ying (GER) (118762),,KIHARA Miyuu (JPN) (131036),,Women's Singles,2,3,4,KIHARA Miyuu,,1018145


In [670]:
name_map = adult.rename(columns={'event_name': 'tournament_name', 'event_type': 'tournament_type', 'event_kind': 'tournament_kind'}).set_index('tournament_name').tournament_id.to_dict()

In [671]:
agamef['tournament_id'] = agamef.tournament_name.map(name_map)

/tmp/ipykernel_27284/3302710675.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agamef['tournament_id'] = agamef.tournament_name.map(name_map)


In [672]:
agamef.to_csv('data/games_2018.tsv', index=False, sep='\t')

In [673]:
agamef

,tournament_name,player_a,player_b,player_x,player_y,event_type,round,res_a,res_x,winner,winner_dbl,match_id,tournament_id
0,"2018 - World Tour, Hungarian Open, Budapest (HUN)",APOLONIA Tiago (POR) (100621),,BADOWSKI Marek (POL) (120154),,Men's Singles,2,4,1,APOLONIA Tiago,,329372,2813
1,"2018 - World Tour, Hungarian Open, Budapest (HUN)",NUYTINCK Cedric (BEL) (106814),,NAGY Krisztian (HUN) (106542),,Men's Singles,2,4,2,NUYTINCK Cedric,,329373,2813
2,"2018 - World Tour, Hungarian Open, Budapest (HUN)",NUYTINCK Cedric (BEL) (106814),,YEFIMOV Viktor (UKR) (110367),,Men's Singles,3,1,4,YEFIMOV Viktor,,329374,2813
3,"2018 - World Tour, Hungarian Open, Budapest (HUN)",YEFIMOV Viktor (UKR) (110367),,NAGY Krisztian (HUN) (106542),,Men's Singles,1,4,1,YEFIMOV Viktor,,329375,2813
4,"2018 - World Tour, Hungarian Open, Budapest (HUN)",FRANZISKA Patrick (GER) (102832),,CHEN Alexander (AUT) (113914),,Men's Singles,2,4,1,FRANZISKA Patrick,,329376,2813
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42630,"2022 - WTT Star Contenders, Doha (QAT)",KARLSSON Kristian (SWE) (104379),,GACINA Andrej (CRO) (102891),,Men's Singles,4,0,4,GACINA Andrej,,1018142,5400
42631,"2022 - WTT Star Contenders, Doha (QAT)",ARUNA Quadri (NGR) (112092),,LIM Jonghoon (KOR) (117357),,Men's Singles,4,2,4,LIM Jonghoon,,1018143,5400
42632,"2022 - WTT Star Contenders, Doha (QAT)",LIN Yun-Ju (TPE) (121582),CHENG I-Ching (TPE) (110797),LEBESSON Emmanuel (FRA) (105136),YUAN Jia Nan (FRA) (137894),Mixed Doubles,2,0,3,LEBESSON Emmanuel,YUAN Jia Nan,1018144,5400
42633,"2022 - WTT Star Contenders, Doha (QAT)",HAN Ying (GER) (118762),,KIHARA Miyuu (JPN) (131036),,Women's Singles,2,3,4,KIHARA Miyuu,,1018145,5400


In [674]:
# agamef = pd.read_csv('data/games_2018.tsv', sep='\t')
agamef.loc[agamef.player_x == 'Mohammed Ali (IND) (133387)', 'player_x'] = 'ALI Mohammed (IND) (133387)'
agamef.loc[agamef.player_x == 'Naina (IND) (119535)', 'player_x'] = 'JAISWAL Naina (IND) (119535)'
agamef.loc[agamef.player_a == 'KHOIRUNISA (INA) (145584)', 'player_a'] = 'KHOIRUNISA Khoirunisa (INA) (145584)'
agamef.loc[agamef.player_x == 'KHOIRUNISA (INA) (145584)', 'player_x'] = 'KHOIRUNISA Khoirunisa (INA) (145584)'
agamef.loc[agamef.player_y == '(INA) (145479)', 'player_y'] = 'GADING PUTRI Az-Hari (INA) (145479)'

In [675]:
players = []

namex = re.compile('(?P<last>[\w\s\'\.-]*[^a-z]) (?P<first>[A-Z][\w\s\.-]+) (?P<year>\(\d+\)\s)?\((?P<country>\w+)\) \((?P<pid>\d+)\)')

for row in agamef.itertuples():
    name = row.player_a.replace('^', '')
    match = namex.match(name)
    if not match:
        print(name, row)
    players.append({
        'player_id': match['pid'],
        'last_name': match['last'],
        'first_name': match['first'],
        'country': match['country'],
    })

    name = row.player_x.replace('^', '')
    match = namex.match(name)
    if not match:
        print(name, row)
    players.append({
        'player_id': match['pid'],
        'last_name': match['last'],
        'first_name': match['first'],
        'country': match['country'],
    })

    if row.player_b and not pd.isna(row.player_b):
        name = row.player_b.replace('^', '')
        match = namex.match(name)
        if not match:
            print(name, row)
        players.append({
            'player_id': match['pid'],
            'last_name': match['last'],
            'first_name': match['first'],
            'country': match['country'],
        })

    if row.player_y and not pd.isna(row.player_y):
        name = row.player_y.replace('^', '')
        match = namex.match(name)
        if not match:
            print(name, row)
        players.append({
            'player_id': match['pid'],
            'last_name': match['last'],
            'first_name': match['first'],
            'country': match['country'],
        })


playerf = pd.DataFrame(players)
playerf.drop_duplicates(inplace=True)

In [676]:
# playerf[playerf.duplicated(subset=['player_id'], keep=False)]
playerf.drop_duplicates(subset=['player_id'], keep='last', inplace=True)
playerf.to_csv('data/players.tsv', index=False, sep='\t')

In [677]:
playerf

,player_id,last_name,first_name,country
0,100621,APOLONIA,Tiago,POR
1,120154,BADOWSKI,Marek,POL
2,106814,NUYTINCK,Cedric,BEL
3,106542,NAGY,Krisztian,HUN
5,110367,YEFIMOV,Viktor,UKR
...,...,...,...,...
87757,144667,ABDULWAHHAB,Abdullah,QAT
87882,134369,SINGEORZAN,Ioana,ROU
88316,134211,ALYAFEI,Khalid,QAT
88453,200413,JOO,Cheonhui,KOR


In [678]:
agamef.drop(columns=['tournament_name', 'winner', 'winner_dbl'], inplace=True)
agamef = agamef.reset_index(drop=True)
agamef

/tmp/ipykernel_27284/423904599.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agamef.drop(columns=['tournament_name', 'winner', 'winner_dbl'], inplace=True)


,player_a,player_b,player_x,player_y,event_type,round,res_a,res_x,match_id,tournament_id
0,APOLONIA Tiago (POR) (100621),,BADOWSKI Marek (POL) (120154),,Men's Singles,2,4,1,329372,2813
1,NUYTINCK Cedric (BEL) (106814),,NAGY Krisztian (HUN) (106542),,Men's Singles,2,4,2,329373,2813
2,NUYTINCK Cedric (BEL) (106814),,YEFIMOV Viktor (UKR) (110367),,Men's Singles,3,1,4,329374,2813
3,YEFIMOV Viktor (UKR) (110367),,NAGY Krisztian (HUN) (106542),,Men's Singles,1,4,1,329375,2813
4,FRANZISKA Patrick (GER) (102832),,CHEN Alexander (AUT) (113914),,Men's Singles,2,4,1,329376,2813
...,...,...,...,...,...,...,...,...,...,...
37672,KARLSSON Kristian (SWE) (104379),,GACINA Andrej (CRO) (102891),,Men's Singles,4,0,4,1018142,5400
37673,ARUNA Quadri (NGR) (112092),,LIM Jonghoon (KOR) (117357),,Men's Singles,4,2,4,1018143,5400
37674,LIN Yun-Ju (TPE) (121582),CHENG I-Ching (TPE) (110797),LEBESSON Emmanuel (FRA) (105136),YUAN Jia Nan (FRA) (137894),Mixed Doubles,2,0,3,1018144,5400
37675,HAN Ying (GER) (118762),,KIHARA Miyuu (JPN) (131036),,Women's Singles,2,3,4,1018145,5400


In [679]:
ids = []

for row in agamef.itertuples():
    id_row = {}
    name = row.player_a.replace('^', '')
    match = namex.match(name)
    if not match:
        print(row)

    id_row['a_id'] = match['pid']

    name = row.player_x.replace('^', '')
    match = namex.match(name)
    if not match:
        print(row)

    id_row['x_id'] = match['pid']

    if row.player_b and not pd.isna(row.player_b):
        name = row.player_b.replace('^', '')
        match = namex.match(name)
        if not match:
            print(row)

        id_row['b_id'] = match['pid']

    if row.player_y and not pd.isna(row.player_y):
        name = row.player_y.replace('^', '')
        match = namex.match(name)
        if not match:
            print(row)

        id_row['y_id'] = match['pid']

    ids.append(id_row)
agamef.drop(columns=['player_a', 'player_b', 'player_x', 'player_y'], inplace=True)
agamef = pd.concat([agamef, pd.DataFrame(ids)], axis = 1)

In [680]:
typ = []
gender = []
for event_type in agamef.event_type:
    if pd.isna(event_type):
        continue

    if 'Men' in event_type:
        gender.append('M')
    elif 'Women' in event_type:
        gender.append('F')
    elif 'Mixed' in event_type:
        gender.append('X')

    if 'Singles' in event_type:
        typ.append('S')
    elif 'Doubles' in event_type:
        typ.append('D')
    elif 'Teams' in event_type:
        typ.append('T')

In [681]:
agamef['gender'] = pd.Series(gender)
agamef['event_type'] = pd.Series(typ)
agamef['round_of'] = agamef['round'].fillna(0).replace('', 0).astype(int)
agamef.drop(columns=['round'], inplace=True)

In [682]:
agamef.loc[20167, 'res_x'] = '4'
agamef.loc[3303, 'res_x'] = '4'

In [683]:
agamef.to_csv('data/games_table_2018.tsv', index=False, sep='\t')

In [684]:
agamef

,event_type,res_a,res_x,match_id,tournament_id,a_id,x_id,b_id,y_id,gender,round_of
0,S,4,1,329372,2813,100621,120154,NaN,NaN,M,2
1,S,4,2,329373,2813,106814,106542,NaN,NaN,M,2
2,S,1,4,329374,2813,106814,110367,NaN,NaN,M,3
3,S,4,1,329375,2813,110367,106542,NaN,NaN,M,1
4,S,4,1,329376,2813,102832,113914,NaN,NaN,M,2
...,...,...,...,...,...,...,...,...,...,...,...
37672,S,0,4,1018142,5400,104379,102891,NaN,NaN,M,4
37673,S,2,4,1018143,5400,112092,117357,NaN,NaN,M,4
37674,D,0,3,1018144,5400,121582,105136,110797,137894,X,2
37675,S,3,4,1018145,5400,118762,131036,NaN,NaN,F,2


In [751]:
mranks = pd.read_csv('data/men2017.csv')
mranks

,Rank,Previous,ID,Assoc,Gender,Name,Points,Previous Points,WeekNum,MonthNum,YearNum
0,1,1,105649,CHN,M,MA Long,3323,3323,0,12,2017
1,2,2,121404,CHN,M,FAN Zhendong,3254,3201,0,12,2017
2,3,3,107028,GER,M,OVTCHAROV Dimitrij,3106,3011,0,12,2017
3,4,0,110267,CHN,M,XU Xin,3078,2994,0,12,2017
4,5,4,101222,GER,M,BOLL Timo,2971,2902,0,12,2017
...,...,...,...,...,...,...,...,...,...,...,...
1465,0,0,120845,KOR,M,KANG Dongsoo,2046,2046,0,12,2017
1466,0,0,109990,GER,M,WANG ^^ Xi,2055,2063,0,12,2017
1467,0,0,117336,JPN,M,JIN Takuya,2123,2131,0,12,2017
1468,0,0,114247,CAN,M,WANG ^ Eugene,2158,2158,0,12,2017


In [686]:
namex2 = re.compile('(?P<last>[\w\'\s\.-]+[^a-z]) (?P<first>[A-Z][\w\s-]+)')

base_players = []
for row in mranks.itertuples():
    name = row.Name.replace('^', '')
    match = namex2.match(name)
    if match is None:
        print(row)
    base_players.append({
        'player_id': str(row.ID),
        'last_name': match['last'],
        'first_name': match['first'],
        'country': row.Assoc,
    })

playerf2 = pd.concat([pd.DataFrame(base_players), playerf])
playerf2['last_name'] = playerf2.last_name.str.strip()
playerf2['first_name'] = playerf2.first_name.str.strip()

In [687]:
playerf2.drop_duplicates(subset=['player_id'], keep='last', inplace=True)
playerf2.reset_index(inplace=True)
playerf2.to_csv('data/players.tsv', index=False, sep='\t')

In [688]:
playerf2.loc[61]

index            462
player_id     101383
last_name      BUKIN
first_name    Andrey
country          RUS
Name: 61, dtype: object

In [689]:
testx = re.compile('(?P<last>[\w\'\s\.-]+[^a-z]) (?P<first>[A-Z][\w\s-]+)')


In [690]:
mpoints = mranks[['ID', 'Points']].set_index('ID').to_dict()['Points']
mpoints

{105649: 3323,
 121404: 3254,
 107028: 3106,
 110267: 3078,
 101222: 2971,
 115910: 2806,
 106195: 2804,
 110729: 2800,
 110553: 2787,
 105197: 2739,
 112062: 2707,
 111606: 2703,
 101820: 2684,
 112620: 2678,
 105926: 2671,
 113059: 2662,
 123980: 2630,
 108246: 2621,
 102841: 2606,
 118761: 2530,
 112092: 2507,
 110411: 2504,
 102761: 2501,
 111595: 2501,
 121752: 2498,
 100696: 2497,
 104379: 2489,
 117113: 2489,
 108581: 2486,
 108958: 2484,
 115641: 2480,
 112409: 2460,
 105136: 2449,
 110730: 2438,
 104257: 2421,
 100621: 2417,
 119588: 2416,
 112074: 2415,
 103107: 2414,
 101648: 2408,
 102697: 2403,
 115439: 2387,
 102380: 2381,
 113165: 2379,
 112580: 2373,
 116620: 2363,
 109946: 2341,
 117357: 2335,
 102832: 2332,
 121412: 2325,
 114936: 2322,
 105339: 2316,
 102891: 2313,
 102968: 2309,
 113336: 2303,
 113931: 2298,
 102943: 2297,
 103037: 2292,
 110410: 2288,
 109459: 2284,
 123690: 2267,
 104314: 2264,
 106296: 2264,
 112735: 2264,
 114355: 2264,
 111050: 2251,
 123962: 2

In [691]:
# assets blackhole, who venmo feed, wallet linking, other ways to show identity privacy, assets volatile, tax parsing

In [692]:
1 / (1 + 10 ** ((3000 - 2800)/400))

0.2402530733520421

In [693]:
scores = {
    (4, 3): 0,
    (4, 2): 0,
    (4, 1): 0,
    (4, 0): 0,
    (3, 2): 0,
    (3, 1): 0,
    (3, 0): 0,
}
for row in agamef.itertuples():
    try:
        if row.res_a == '4':
            scores[(int(row.res_a), int(row.res_x))] += 1
        elif row.res_x == '4':
            scores[(int(row.res_x), int(row.res_a))] += 1
        elif row.res_a == '3':
            scores[(int(row.res_a), int(row.res_x))] += 1
        elif row.res_x == '3':
            scores[(int(row.res_x), int(row.res_a))] += 1
        # else:
        #     print(row)
    except:
        print(row)
        raise
scores

{(4, 3): 2891,
 (4, 2): 3818,
 (4, 1): 5091,
 (4, 0): 6568,
 (3, 2): 3868,
 (3, 1): 5903,
 (3, 0): 9447}

In [793]:
agamef.res_a = agamef.res_a.astype(int)
agamef.res_x = agamef.res_x.astype(int)

In [714]:
mpoints2 = {
    str(k): math.floor(v/2)
    for k, v in mpoints.items()
}
mpoints2

{'105649': 1661,
 '121404': 1627,
 '107028': 1553,
 '110267': 1539,
 '101222': 1485,
 '115910': 1403,
 '106195': 1402,
 '110729': 1400,
 '110553': 1393,
 '105197': 1369,
 '112062': 1353,
 '111606': 1351,
 '101820': 1342,
 '112620': 1339,
 '105926': 1335,
 '113059': 1331,
 '123980': 1315,
 '108246': 1310,
 '102841': 1303,
 '118761': 1265,
 '112092': 1253,
 '110411': 1252,
 '102761': 1250,
 '111595': 1250,
 '121752': 1249,
 '100696': 1248,
 '104379': 1244,
 '117113': 1244,
 '108581': 1243,
 '108958': 1242,
 '115641': 1240,
 '112409': 1230,
 '105136': 1224,
 '110730': 1219,
 '104257': 1210,
 '100621': 1208,
 '119588': 1208,
 '112074': 1207,
 '103107': 1207,
 '101648': 1204,
 '102697': 1201,
 '115439': 1193,
 '102380': 1190,
 '113165': 1189,
 '112580': 1186,
 '116620': 1181,
 '109946': 1170,
 '117357': 1167,
 '102832': 1166,
 '121412': 1162,
 '114936': 1161,
 '105339': 1158,
 '102891': 1156,
 '102968': 1154,
 '113336': 1151,
 '113931': 1149,
 '102943': 1148,
 '103037': 1146,
 '110410': 114

In [1139]:
from collections import defaultdict

provis = defaultdict(lambda: 0)
provis_count = defaultdict(lambda: 0)

K = 40
PROV = 200
PROV_THRESH = 10
PROV_PER_SET = False
CARRY = 1

mpoints2 = {
    str(k): math.floor(v * CARRY)
    for k, v in mpoints.items()
}

correct = 0
total = 0

active_players = set()

for game in agamef[pd.isna(agamef.b_id) & (agamef.gender == 'M')].itertuples():            
    active_players.add(game.a_id)
    active_players.add(game.x_id)
    rating_a = mpoints2.get(game.a_id)
    rating_x = mpoints2.get(game.x_id)
    a_win = game.res_a > game.res_x
    win_sets = abs(game.res_a - game.res_x)

    if PROV_PER_SET:
        prize = (1 - 0.1 * win_sets) * PROV
    else:
        prize = PROV
    if rating_a is None and rating_x is None:
        # double provisional game
        if a_win:
            provis[game.a_id] += prize / 2
            provis[game.x_id] -= prize / 2
        else:
            provis[game.x_id] += prize / 2
            provis[game.a_id] -= prize / 2
        provis_count[game.a_id] += 1
        provis_count[game.x_id] += 1
        if provis_count[game.a_id] == PROV_THRESH:
            mpoints2[game.a_id] = provis[game.a_id]
            del provis[game.a_id]
        if provis_count[game.x_id] == PROV_THRESH:
            mpoints2[game.x_id] = provis[game.x_id]
            del provis[game.x_id]
    elif rating_a is None:
        if a_win:
            provis[game.a_id] += prize
        else:
            provis[game.a_id] -= prize
        provis_count[game.a_id] += 1
        if provis_count[game.a_id] == PROV_THRESH:
            mpoints2[game.a_id] = provis[game.a_id]
            del provis[game.a_id]
    elif rating_x is None:
        if a_win:
            provis[game.x_id] -= prize
        else:
            provis[game.x_id] += prize
        provis_count[game.x_id] += 1
        if provis_count[game.x_id] == PROV_THRESH:
            mpoints2[game.x_id] = provis[game.x_id]
            del provis[game.x_id]
    else:
        # normal game
        diff = (rating_x - rating_a) / 400
        exa = 1 / (1 + 10 ** diff)
        if a_win:
            points = K * (1 - exa)
            mpoints2[game.a_id] += points
            mpoints2[game.x_id] -= points
            if diff < 0:
                correct += 1
        else:
            points += K * exa
            mpoints2[game.x_id] += points
            mpoints2[game.a_id] -= points
            if diff > 0:
                correct += 1
        if diff != 0:
            total += 1
(correct / total) * 100

72.34892113952239

In [1140]:
plup = playerf2.set_index('player_id').to_dict('index')

In [1141]:
ranks = []
for k in active_players:
    player = plup.get(k)
    v = mpoints2.get(k)
    if not v:
        continue
    ranks.append(('{last_name} {first_name} {country}'.format(**player), math.ceil(v)))
ranks.sort(key=lambda x: -x[1])
print('\n'.join([f"{x[0]} {x[1]}" for x in ranks[:100]]))

XU Xin CHN 3804
MA Long CHN 3792
FAN Zhendong CHN 3652
LIN Gaoyuan CHN 3380
LIANG Jingkun CHN 3344
WANG Chuqin CHN 3310
BOLL Timo GER 3197
LIN Yun-Ju TPE 3167
OVTCHAROV Dimitrij GER 3132
HARIMOTO Tomokazu JPN 3027
JEOUNG Youngsik KOR 2978
CALDERANO Hugo BRA 2972
MIZUTANI Jun JPN 2881
JORGIC Darko SLO 2818
ZHOU Yu CHN 2817
CHO Seungmin KOR 2804
ZHANG Jike CHN 2784
YAN An CHN 2756
LIU Dingshuo CHN 2711
QIU Dang GER 2703
FRANZISKA Patrick GER 2695
YOSHIMURA Maharu JPN 2682
LIM Jonghoon KOR 2681
MOREGARD Truls SWE 2647
DUDA Benedikt GER 2644
KARLSSON Kristian SWE 2626
ZHAO Zihao CHN 2577
SHIBAEV Alexander RUS 2572
UDA Yukiya JPN 2571
GACINA Andrej CRO 2563
JANG Woojin KOR 2557
PITCHFORD Liam GBR 2548
FREITAS Marcos POR 2533
KALLBERG Anton SWE 2531
JEONG Sangeun KOR 2510
MORIZONO Masataka JPN 2499
FANG Bo CHN 2482
GNANASEKARAN Sathiyan IND 2463
LEE Sangsu KOR 2454
OSHIMA Yuya JPN 2453
WONG Chun Ting HKG 2449
CHUANG Chih-Yuan TPE 2449
JHA Kanak USA 2442
ARUNA Quadri NGR 2440
TOGAMI Shunsuke 

In [1040]:
agamef[pd.isna(agamef.b_id)]

,event_type,res_a,res_x,match_id,tournament_id,a_id,x_id,b_id,y_id,gender,round_of
0,S,4,1,329372,2813,100621,120154,NaN,NaN,M,2
1,S,4,2,329373,2813,106814,106542,NaN,NaN,M,2
2,S,1,4,329374,2813,106814,110367,NaN,NaN,M,3
3,S,4,1,329375,2813,110367,106542,NaN,NaN,M,1
4,S,4,1,329376,2813,102832,113914,NaN,NaN,M,2
...,...,...,...,...,...,...,...,...,...,...,...
37671,S,0,4,1018141,5400,114105,131036,NaN,NaN,F,4
37672,S,0,4,1018142,5400,104379,102891,NaN,NaN,M,4
37673,S,2,4,1018143,5400,112092,117357,NaN,NaN,M,4
37675,S,3,4,1018145,5400,118762,131036,NaN,NaN,F,2
